<a href="https://colab.research.google.com/github/anirudhakolay/Text_to_ImageGeneratorModel/blob/main/Gradio_Text_to_Image_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# First, make sure you have all the necessary libraries installed:
# pip install torch diffusers accelerate gradio

import torch
from diffusers import StableDiffusionPipeline
import gradio as gr

# --- 1. Load the AI Model ---
# We load the model only once when the script starts to avoid reloading it
# for every user request, which would be very slow.
print("Loading Stable Diffusion model... This may take a moment.")

# --- OPTIMIZATION: Using a smaller, faster model designed for speed ---
# This model is much faster on CPUs than the standard Stable Diffusion 1.5
model_id = "segmind/tiny-sd"

# We use float16 for memory efficiency.
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

# Check if a CUDA-enabled GPU is available and move the model to the GPU
# for much faster image generation.
if torch.cuda.is_available():
    print("GPU detected. Moving model to CUDA.")
    pipe = pipe.to("cuda")
else:
    print("Warning: CUDA not available. Running on CPU. Performance will be slower.")
    # For CPU, it's better to work with full precision.
    pipe.to(torch.float32)


# --- 2. Define the Image Generation Function ---
# This is the core function that takes user inputs and returns an image.
def generate_image(prompt, negative_prompt, steps):
    """
    Takes a text prompt, a negative prompt, and the number of inference steps,
    and uses the Stable Diffusion model to generate an image.
    """
    print(f"Received prompt: '{prompt}'")
    print(f"Negative prompt: '{negative_prompt}'")
    print(f"Inference steps: {steps}")

    # The 'pipe' object is called like a function. It handles the whole
    # diffusion process. We access the 'images' list and take the first one.
    try:
        # Pass the negative_prompt and num_inference_steps to the pipeline
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps
        ).images[0]

        print("Image generated successfully.")
        return image
    except Exception as e:
        print(f"An error occurred during image generation: {e}")
        return None


# --- 3. Create the Gradio Web Interface ---
# We use gr.Blocks() for more control over the layout.
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # AI Text-to-Image Generator (Optimized)
        This version uses a faster model and lets you control generation steps for better performance on a CPU.
        """
    )

    with gr.Row():
        with gr.Column(scale=4):
            # The Textbox is the input field for the user's prompt.
            prompt_input = gr.Textbox(
                label="Your Prompt",
                placeholder="e.g., A majestic lion wearing a crown, photorealistic",
                lines=2
            )
            negative_prompt_input = gr.Textbox(
                label="Negative Prompt (what to avoid)",
                placeholder="e.g., blurry, ugly, deformed, bad anatomy",
                lines=2
            )
        with gr.Column(scale=1):
             # The button that the user will click to start the generation.
            submit_button = gr.Button("Generate Image", variant="primary", scale=1)

    # --- OPTIMIZATION: Add a slider to control speed vs. quality ---
    steps_slider = gr.Slider(
        minimum=10,
        maximum=50,
        step=1,
        value=20,
        label="Inference Steps",
        info="Fewer steps are faster but may reduce quality. More steps are slower but can improve detail."
    )

    # The Image component will display the output from our function.
    output_image = gr.Image(label="Generated Image")

    # --- 4. Connect the Components ---
    # This is the event listener. When the 'submit_button' is clicked,
    # it calls the 'generate_image' function with the inputs
    # and puts the result into 'output_image'.
    submit_button.click(
        fn=generate_image,
        inputs=[prompt_input, negative_prompt_input, steps_slider],
        outputs=output_image
    )

# --- 5. Launch the Application ---
# The launch() method starts a local web server.
print("Launching Gradio interface... Open the local URL in your browser.")
demo.launch()


Loading Stable Diffusion model... This may take a moment.


model_index.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--tiny-sd/snapshots/cad0bd7495fa6c4bcca01b19a723dc91627fe84f/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--tiny-sd/snapshots/cad0bd7495fa6c4bcca01b19a723dc91627fe84f/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--tiny-sd/snapshots/cad0bd7495fa6c4bcca01b19a723dc91627fe84f/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--tiny-sd/snapshots/cad0bd7495fa6c4bcca01b19a723dc91627fe84f/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Launching Gradio interface... Open the local URL in your browser.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e0ccc50caada54564.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
! pip install diffusers accelerate gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00
